In [ ]:
# Import libraries
import numpy as np
import MDAnalysis as mda
import nmrformd as nmrmd

recalculate=False

# For figure plotting
import sys
from matplotlib import pyplot as plt
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
msize = 15
fontsize = 30
font = {'family': 'sans', 'color':  'black', 'weight': 'normal', 'size': fontsize}
myblue_light = [42/ 255, 99/255, 255/ 255] # [0/ 255, 150/255, 177/ 255]
myblue_dark = [43/ 255, 140/255, 238/ 255] # [0/ 255, 150/255, 177/ 255]
myred= [244/ 255, 143/255, 97/ 255] # [255/ 255, 77/255, 77/ 255]
mygreen = [122/255, 190/255, 165/255]
lightgray = [0.1, 0.1, 0.1]
darkgray = [0.9, 0.9, 0.9]
white = [0, 0, 0]
black = [1, 1, 1]
sys.path.append('../../shared/')
from tools import figure_improvement, save_figure

# Path to data
datapath = "../../raw-data/bulk-water/N398/"

if recalculate:
    # Create a MDAnalysis universe
    u = mda.Universe(datapath+"topology.data", datapath+"prod.xtc")
    #u.transfer_to_memory(stop=50) # to analyse only a subpart of the trajectory

    # Print a few information
    n_molecules = u.atoms.n_residues
    print(f"The number of water molecules is {n_molecules}")
    timestep = np.int32(u.trajectory.dt)
    print(f"The timestep is {timestep} ps")
    total_time = np.int32(u.trajectory.totaltime)
    print(f"The total simulation time is {total_time} ps")

    # cut-off for frequency and time
    f_1 = 1
    t_1 = 2
    if u.trajectory.n_frames == 10001:
        f_1 = 2600
        t_1 = 9000

    # Run NMRforMD
    group_i = u.select_atoms("type 2")
    nmr_result = nmrmd.NMR(u, group_i, number_i=80) # 3-4 minutes to complete

    f, T1 = nmr_result.f[:-f_1], 1/nmr_result.R1[:-f_1]
    np.savetxt("../../treated-data/tutorials/bulk-water/T1.dat", np.vstack([f, T1]).T)
    t, gij = nmr_result.t[:-t_1], nmr_result.gij[0][:-t_1]
    np.savetxt("../../treated-data/tutorials/bulk-water/gij.dat", np.vstack([t, gij]).T)

    # Separate intra and inter motion
    nmr_result_intra = nmrmd.NMR(u, group_i, type_analysis="intra_molecular", number_i=120) # 2-3 minutes
    nmr_result_inter = nmrmd.NMR(u, group_i, type_analysis="inter_molecular", number_i=80) # 3-4 minutes
    f, T1_intra = nmr_result_intra.f[:-f_1], 1/nmr_result_intra.R1[:-f_1]
    f, T1_inter = nmr_result_inter.f[:-f_1], 1/nmr_result_inter.R1[:-f_1]
    np.savetxt("../../treated-data/tutorials/bulk-water/T1_intra.dat", np.vstack([f, T1_intra]).T)
    np.savetxt("../../treated-data/tutorials/bulk-water/T1_inter.dat", np.vstack([f, T1_inter]).T)
    t, gij_intra = nmr_result_intra.t[:-t_1], nmr_result_intra.gij[0][:-t_1]
    t, gij_inter = nmr_result_inter.t[:-t_1], nmr_result_inter.gij[0][:-t_1]
    np.savetxt("../../treated-data/tutorials/bulk-water/gij_intra.dat", np.vstack([t, gij_intra]).T)
    np.savetxt("../../treated-data/tutorials/bulk-water/gij_inter.dat", np.vstack([t, gij_inter]).T)
else:
    f, T1 = np.loadtxt("../../treated-data/tutorials/bulk-water/T1.dat").T
    t, gij = np.loadtxt("../../treated-data/tutorials/bulk-water/gij.dat").T
    f, T1_intra = np.loadtxt("../../treated-data/tutorials/bulk-water/T1_intra.dat").T
    f, T1_inter = np.loadtxt("../../treated-data/tutorials/bulk-water/T1_inter.dat").T
    t, gij_intra = np.loadtxt("../../treated-data/tutorials/bulk-water/gij_intra.dat").T
    t, gij_inter = np.loadtxt("../../treated-data/tutorials/bulk-water/gij_inter.dat").T

# T1 bulk water
for mode, mygray, edge, myblue in zip(['light', 'dark'], [lightgray, darkgray], [black, white], [myblue_light, myblue_dark]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(f, T1, 'o', markersize=msize, markeredgecolor=edge, markeredgewidth=0.2, color=myblue)
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$f$ (MHz)", ylabel=r'$T_1$ (s)',
                       xlim=(1e2, 2e5), ylim=(1, 200))
    save_figure(plt, fig, mode, name='tutorials/bulk-water/T1', save=True)

# Correlation function
for mode, mygray, edge, myblue in zip(['light', 'dark'], [lightgray, darkgray], [black, white], [myblue_light, myblue_dark]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(t, gij/gij[0], 'o', markersize=msize, markeredgecolor=edge, markeredgewidth=0.2, color=myblue)
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$t$ (ps)", ylabel=r'$G^{(0)} (t) / G^{(0)} (0)$',
                       xlim=(1, 200), ylim=(0.0001, 2))
    save_figure(plt, fig, mode, name='tutorials/bulk-water/Gij', save=True)

for mode, mygray, edge, myblue in zip(['light', 'dark'], [lightgray, darkgray], [black, white], [myblue_light, myblue_dark]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(f, 1/T1, 'o', markersize=msize, markeredgecolor=edge, markeredgewidth=0.2, color=myblue, label="Total")
    ax1.loglog(f, 1/T1_intra, '^', markersize=msize, color=myred, markeredgecolor=edge, markeredgewidth=0.2, label="Intra-molecular")
    ax1.loglog(f, 1/T1_inter, 's', markersize=msize, color=mygreen, markeredgecolor=edge, markeredgewidth=0.2, label="Inter-molecular")
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$f$ (MHz)", ylabel=r'$R_1$ (s$^{-1}$)',
                       xlim=(1e2, 2e5), ylim=(0.01, 1))
    save_figure(plt, fig, mode, name='tutorials/bulk-water/R1-intra-inter', save=True)

for mode, mygray, edge, myblue in zip(['light', 'dark'], [lightgray, darkgray], [black, white], [myblue_light, myblue_dark]):
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    #ax1.loglog(nmr_result.t, nmr_result.gij[0]/nmr_result.gij[0][0], 'o', markersize=8, color=myblue, label="Total")
    ax1.loglog(t, gij_intra/gij_intra[0], '^', markersize=msize, color=myred, markeredgecolor=edge, markeredgewidth=0.5, label="Intra-molecular")
    ax1.loglog(t, gij_inter/gij_inter[0], 's', markersize=msize, color=mygreen, markeredgecolor=edge, markeredgewidth=0.5, label="Inter-molecular")
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$t$ (ps)", ylabel=r'$G^{(0)} (t) / G^{(0)} (0)$',
                       xlim=(1, 500), ylim=(0.00001, 1))
    save_figure(plt, fig, mode, name='tutorials/bulk-water/Gij-intra-inter', save=True)